# gerando a base de dados

In [3]:
import pygame
import pymunk
import pymunk.pygame_util
import random
from math import sin, cos



pygame 2.1.2 (SDL 2.0.16, Python 3.8.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


## -nessa simulação serão usados o Pygame para gerar os graficos, e o Pymunk para fazer os calculculos de fisica bidimensional

In [4]:
color =[(0,0,0,255),(200,15,15,255),(15,200,15,255),(15,15,200,255),(100,10,100,255)]

def create_body(): #cria o corpo, entidade responsavel por armazenar informaçoes gerais da entidade, como posição inicial, massa, etc...
    body = pymunk.Body(mass=1, moment=10)
    body.position = 512+random.randint(-100,100), 200
    print("made a body")
    return body

def create_shape(body,size): # cria a forma de uma bola associada a um corpo
    shape = pymunk.Circle(body,radius=size)
    shape.color = (255, 0, 0, 255)
    print("made a ball")
    return shape

def creatething():
    body = pymunk.Body(mass=1, moment=10)
    body.position = 512+random.randint(-462, 462), 0
    circle = pymunk.Circle(body, radius=60)
    circle.color = color[random.randint(0,4)]
    circle.elasticity = 0.2
    return body, circle

def createconesec(): #cria um trapézio
    body = pymunk.Body(mass=1, moment=10)
    body.position = 512+random.randint(-462, 462), 0
    body.angle = (random.randint(0, 360))*0.0174533
    
    w, h = 125, int(125*(4/5))
    vs = [(-63,-30), (63,-50), (63,50), (-63,30)]
    circle = pymunk.Poly(body, vs)
    circle.color = color[random.randint(0,4)]
    circle.elasticity = 0.00001
    circle.friction=0.4
    circle.density=0.0001
    return body, circle

def createhalfcircle(): #cria um semi circulo
    body = pymunk.Body(mass=1, moment=10)
    body.position = 512+random.randint(-462, 462), 0
    body.angle = (random.randint(0, 360))*0.0174533
    
    radius=90
    segs=32
    vs=[]
    for x in range(segs):
        frac=2*3.14159/(segs+1)
        if cos(frac*x)<0:
            vs.append((0,sin(frac*x)*radius))
        else:
            vs.append((cos(frac*x)*radius,sin(frac*x)*radius))
            
    
    circle = pymunk.Poly(body, vs)
    circle.color = color[random.randint(0,4)]
    circle.elasticity = 0.00001
    circle.friction=0.4
    circle.density=0.0001
    return body, circle

def createmoon(): #cria um circulo com 1/3 faltando, que chamaos de "lua"
    body = pymunk.Body(mass=1, moment=10)
    body.position = 512+random.randint(-462, 462), 0
    body.angle = (random.randint(0, 360))*0.0174533
    
    radius=70
    segs=32
    vs=[]
    vs2=[(0,0)]
    for x in range(segs):
        frac=2*3.14159/(segs+1)
        if cos(frac*x)<0:
            vs.append((0,sin(frac*x)*radius))
            if sin(frac*x)>0:
                vs2.append((cos(frac*(x+1)+1.5708)*radius,sin(frac*(x+1)+1.5708)*radius))
        else:
            vs.append((cos(frac*x)*radius,sin(frac*x)*radius))
            
    circle = pymunk.Poly(body, vs)
    circle2 = pymunk.Poly(body, vs2)
    circlecolor = color[random.randint(0,4)]
    circle.color = circlecolor
    circle.elasticity = 0.00001
    circle.friction=0.4
    circle.density=0.0001
    circle2.color = circlecolor
    circle2.elasticity = 0.00001
    circle2.friction=0.4
    circle2.density=0.0001
    return body, circle, circle2
    

## agora que as funções responsaveis por criar os corpos estão definidas podemos começar a simulação

In [ ]:
pygame.init()

display = pygame.display.set_mode((1024,1024)) #cria o display do pygame, para que a fisica possa ser vizualizada
draw_options = pymunk.pygame_util.DrawOptions(display) #salva todas as posições, formas, cores, para que o pygame possa atualizar os quadros
clock = pygame.time.Clock()
space = pymunk.Space() #cria o espaçõ fisico onde a simulação será calculada
space.gravity = 0,9000 #atribui uma força que puxa os objetos no sentido em que Y cresce, Obs.: no pygame o ponto (0,0) fica no canto superior esquerdo, X cresce para a direita e Y cresce para baixo
FPS = 1000 #variavel que nos permite controlar a velocidade da simulação
bs = [] #lista onde serão armazenados os corpos criados
ss = [] #lista onde serão armazenados as formas criadas
tot = 0 #variavel usada para rastrear o numero de objetos em uma caixa
cc = 0 #contador usado para anotar o numero da caixa
c = 0 #contador usado para dar delay entre as fotos
F = False #variavel que anota se a "foto" já foi tirada ou não
caixa = 0 #contador que armazena o numero de caixas geradas

b0 = space.static_body #corpo estatico é um objeto que náo é afetado por forças externas
chao = pymunk.Segment(b0, (0, 1024), ( 1024,  1024), 1)
chao.elasticity = 0.0
chao.friction=0.4
parede = pymunk.Segment(b0, (0, 0), (0,  1024), 1)
parede.elasticity = 0.0
parede.friction=0.4
parede1 = pymunk.Segment(b0, ( 1024, 0),  (1024,  1024), 1)
parede1.elasticity = 0.0
parede1.friction=0.4

body, circle = creatething()

space.add(chao,parede,parede1)


running = True
while running:
    for event in pygame.event.get(): #permite interação do usuario
        if event.type == pygame.QUIT: #faz com que o usuario possa fechar a simulação a qualquer momento
            running = False
    if c ==400 and tot != 51: #checa se ja passou tempo sufuciente des de que o ultimo objeto foi adicionado a caixa
        kinsum=0.0
        for bodeh in bs:
            kinsum=bodeh.kinetic_energy+kinsum #cria uma variavel que soma toda a energia dentro da simulação
        
        #if kinsum < 0.0000000002*(tot+1):
        if kinsum < 0.02*(tot+1): #só permite que a foto seja tirada se os corpos estiverem quase parados
            pygame.image.save(display, "provisorio/caixa{}_qtd{}.png".format(str(caixa).zfill(3),str(tot).zfill(2))) #salva uma imagem no formato "caixa###_qtd###"
            F = True

        else: 
            c=200

    display.fill((255,255,255)) #dá um fundo branco a simulação
    space.debug_draw(draw_options)
    pygame.display.update() #atualizã o quadro dentro da janela da simulação
    clock.tick(FPS)
    space.step(1/FPS)
    if F == True: #assim que a foto é tirada, a simulação cria o proximo objeto
        for bodeh in bs: #uma força aleatoria é colocada em todos os corpos para simular a caixa sendo chacoalhada
            impulse = (random.randint(-1000, 1000), random.randint(-1000, 1000))
            bodeh.apply_impulse_at_local_point(impulse)
            bodeh._set_torque((random.randint(-5, 5))*0.0174533)
            
        #body1, circle1 = createconesec()
        #body1, circle1 = createhalfcircle()
        body1, circle1, circle2 = createmoon()
        bs.append(body1)
        ss.append(circle1)
        ss.append(circle2)
        space.add(bs[-1],ss[-1],ss[-2])
        tot = tot + 1
        F = False
        c =0
    c = c + 1
    if tot == 51: #quando a caixa tiver 50 itens eles são apagados e a proxima cixa começa a ser preenchida
        for i in range(len(bs)):
            space.remove(bs[i])
            space.remove(ss[i])
        bs = []
        ss = []
        tot = 0
        caixa = caixa +1
    if caixa ==51: #quando 50 caixas forem feitas a simulação para
    #if caixa ==1:
        running = False


NameError: name 'pygame' is not defined